In [1]:
%pip install youtube-transcript-api
%pip install groq
%pip install flask
%pip install googlesearch-python
%pip install duckduckgo-search
%pip install youtube-search-python
%pip install langchain-experimental

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [22]:
from googlesearch import search
from openai import OpenAI
from youtube_transcript_api import YouTubeTranscriptApi
from flask import Flask, request, jsonify
from duckduckgo_search import AsyncDDGS, DDGS
import requests
from bs4 import BeautifulSoup
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from groq import Groq
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma


app = Flask(__name__)
client = OpenAI(api_key="sk-proj-t8zmYeKLmyl7Jj1JuCTsT3BlbkFJYNmTpjCTi05I0So7HHOC")
groqclient=Groq(api_key="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")


In [25]:
transcript = YouTubeTranscriptApi.get_transcript("GLKoDkbS1Cg")
transcription=""
for i in transcript:
    transcription+=i["text"]
print(transcription)

[Music]welcome to the entrepreneurial thoughtleader seminar at StanfordUniversity this is the Stanford seminarfor aspiring entrepreneurs ETL isbrought to you by stvp the Stanfordentrepreneurship engineering center andbasis The Business Association ofStanford entrepreneurial students I'mrvie balani a lecturer in the managementscience and engineering department andthe director of Alchemist andaccelerator for Enterprise startups andtoday I have the pleasure of welcomingSam Altman to ETLum Sam is the co-founder and CEO of openAI open is not a word I would use todescribe the seats in this class and soI think by virtue of that that everybodyalready play knows open AI but for thosewho don't openai is the research anddeployment company behind chat gbt Dollyand Sora um Sam's life is a pattern ofbreaking boundaries and transcendingwhat's possible both for himself and forthe world he grew up in the midwest inSt Louis came to Stanford took ETL as anundergrad um for any and we we held onto Stanford

In [26]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "system",
            "content": """extract all points (make sure each point has full context of the data provided. dont use pronouns) from the data as they are and split each point with delimiter '@@@' return the response within a codeblock  for example -         
```point1@@@
point2@@@
point3@@@
...```"""
        },
        {
            "role": "user",
            "content": transcription
        }
    ],
    temperature=1,
    max_tokens=4096,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)
points_list=response.choices[0].message.content

In [58]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "system",
            "content": """extract all points which can be answered true or false which can be used for factual validation and dont include points like introductory statements or anything that is not topic specific or content specific (make sure each point has full context of the data provided. dont use pronouns do not use pronouns DO NOT USE PRONOUNS) from the data as they are and add '@@@' after each point and return the response within a codeblock"""
        },
        {
            "role": "user",
            "content": transcription
        }
    ],
    temperature=1,
    max_tokens=4096,
    top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)
points_list=response.choices[0].message.content
print(points_list)

```
1. Sam Altman is the co-founder and CEO of openAI. @@@
2. OpenAI is a nonprofit research lab with the mission to build general purpose artificial intelligence that benefits all of Humanity. @@@
3. OpenAI set the record for the fastest growing app in history with the launch of chat GPT, growing to 100 million active users just two months after launch. @@@
4. Sam was named one of Times's 100 most influential people in the world. @@@
5. Sam's life is a pattern of breaking boundaries and transcending what's possible for himself and the world. @@@
6. Sam co-founded openAI in 2015. @@@
7. Sam has a husband and splits his time between San Francisco and Napa. @@@
8. Resilience can be taught and will be more important in the coming decades, according to Sam. @@@
9. Sam believes that Fusion will eventually dominate electrical generation on Earth. @@@
10. Sam thinks that AI will be a net good, even though the technology can be misused. @@@
```


In [52]:
points_list=points_list[points_list.index('```')+3:points_list.rfind('```')].strip()
points_list=points_list.replace('\n',"")

In [53]:
points_list=points_list.split('@@@')

In [54]:
for point in points_list:
    print(point)

- OpenAI is the research and deployment company behind chat gbt Dolly and Sora. 
- Sam Altman is the co-founder and CEO of OpenAI, a nonprofit research lab with the mission to build general purpose artificial intelligence that benefits all Humanity. 
- OpenAI set the record for the fastest growing app in history with the launch of chat gbt which grew to 100 million active users just two months after launch. 
- Sam Altman studied computer science at Stanford and joined the inaugural class of Y combinator with a Social Mobile app company called looped. 
- Sam Altman became the president of Y Combinator from 2014 to 2019. 
- Sam Altman co-founded openAI as a nonprofit research lab with the mission to build general purpose artificial intelligence that benefits all Humanity. 
- OpenAI has a structure where the nonprofit owns the for-profit part of the organization. 
- The culture at OpenAI is driven by a shared mission and deep sense of purpose. 
- AI infrastructure is predicted to be one o

In [55]:
for i in points_list:
    if i=='':
        points_list.remove(i)

In [56]:
print(points_list)
print(len(points_list))

['- OpenAI is the research and deployment company behind chat gbt Dolly and Sora. ', '- Sam Altman is the co-founder and CEO of OpenAI, a nonprofit research lab with the mission to build general purpose artificial intelligence that benefits all Humanity. ', '- OpenAI set the record for the fastest growing app in history with the launch of chat gbt which grew to 100 million active users just two months after launch. ', '- Sam Altman studied computer science at Stanford and joined the inaugural class of Y combinator with a Social Mobile app company called looped. ', '- Sam Altman became the president of Y Combinator from 2014 to 2019. ', '- Sam Altman co-founded openAI as a nonprofit research lab with the mission to build general purpose artificial intelligence that benefits all Humanity. ', '- OpenAI has a structure where the nonprofit owns the for-profit part of the organization. ', '- The culture at OpenAI is driven by a shared mission and deep sense of purpose. ', '- AI infrastructur

In [57]:
for point in points_list:
    print(point)
    results = await AsyncDDGS().news(point, region='wt-wt', safesearch='off', timelimit='m', max_results=1)
    loader = WebBaseLoader(results[0]["url"])
    docs = loader.load()
    

    response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "system",
            "content": """you are a fact validator, your task is to check for statement[] in resource[] and return only true or false (lowercase) one word"""
        },
        {
            "role": "user",
            "content": f"statement[{point}],resource[{docs}]"
        }
    ],
        temperature=1,
  max_tokens=256,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
    )
    relornot=response.choices[0].message.content
    print(relornot)
    print()
    print()

- OpenAI is the research and deployment company behind chat gbt Dolly and Sora. 
false


- Sam Altman is the co-founder and CEO of OpenAI, a nonprofit research lab with the mission to build general purpose artificial intelligence that benefits all Humanity. 
true


- OpenAI set the record for the fastest growing app in history with the launch of chat gbt which grew to 100 million active users just two months after launch. 
false


- Sam Altman studied computer science at Stanford and joined the inaugural class of Y combinator with a Social Mobile app company called looped. 
false


- Sam Altman became the president of Y Combinator from 2014 to 2019. 
false


- Sam Altman co-founded openAI as a nonprofit research lab with the mission to build general purpose artificial intelligence that benefits all Humanity. 
false


- OpenAI has a structure where the nonprofit owns the for-profit part of the organization. 
false


- The culture at OpenAI is driven by a shared mission and deep sense of